In [1]:
import requests
from bs4 import BeautifulSoup as BSoup
import pandas as pd

In [2]:
import csv
import requests
import os
from datetime import datetime, timedelta
from abc import ABC, abstractmethod
import traceback

class SecFiling(ABC):
    """Holds the data of a generic SEC filing.
    
    Expected to be subclassed by 'SecFiling10K' and 'SecFiling10Q'. Also expected to be a utility to CanslimParams, i.e.
    driven by it.
    
    Before running this, make sure my edgar_idx.db is up-to-date. analyze_stockdata.get_list_sec_filings() will 
    update the 'idx' table (i.e. the database with the relevant urls), and analyze_stockdata.get_cik_ticker_lookup_db()
    will update the lookup table for CIK number - ticker symbol correlation.
    """
    def __init__(self, ticker):
        self.ticker = ticker
        self.xbrlInstance = None
        self.contextIds = {}
        self.currentContextId = ""
        self.errorLog = []
        
        
    def __del__(self):
        self.contextIds.clear()
        del self.errorLog[:]
        
        
    def download(self, cik, co_name, filing_type, filing_date, filing_link):
        """Retrieves a SEC filing from EDGAR. Follow by a call to 'load()'.
        
        Also a roundabout way of getting the filename for this data file.
        """
        # inspired by http://kaikaichen.com/?p=681
        saveas = '_'.join([cik, co_name, filing_type, filing_date])
        saveDir = os.path.join("SECDATA", co_name)
        self.fname = os.path.join (saveDir, saveas)
        ## Only download if the file doesn't exist yet (at the expected path)
        if not os.path.exists(self.fname):
            url = 'https://www.sec.gov/Archives/' + filing_link.strip()
            if not os.path.exists(saveDir):
                os.makedirs(saveDir)
            with open("logfile.txt", "a+") as logfile:
                with open(self.fname, 'wb') as f:
                    f.write(requests.get('%s' % url).content)
                    logfile.write('{:s} - downloaded and saved as {:s}\n'.format(url, self.fname))
        return (self.fname)
    
    
    def load(self, fname):
        """Loads the data of SEC filing."""
        f = open (fname, "r")
        ## TODO: Do I need to have the whole file in memory?
        wholeFile = f.read ()
        f.close ()
        soup = BSoup (wholeFile, "lxml")
        ## Extract some basic info from the SEC-header section
        sec_header = soup.find("sec-header")
        try:
            for l in sec_header.text.split("\n"):
                if 'conformed period of report' in l.lower():
                    self.reportDate = datetime.strptime((l.split(":")[1]).strip(), "%Y%m%d")
                if 'standard industrial classification' in l.lower():
                    self.stdIndustrialClass = l.split(":")[1].strip()
            print("Report date: {:s}, industrial class: {:s}". format(str(self.reportDate), self.stdIndustrialClass))
        except:
            self.errorLog.append("Unable to find SEC-header in file.")
            return False
        
        ## Find the XBRL Instance document. It has the description-tag "XBRL INSTANCE DOCUMENT" within its document-tag.
        doc_tag = soup.find_all("document")
        try:
            for tag in doc_tag:
                description = tag.find("description").get_text().lower()
                if (description.startswith("xbrl instance document")):
                    self.xbrlInstance = tag
                    break
        except:
            self.errorLog.append("Unable to find document-tags in file.")
            return False
        
        if not self.xbrlInstance:
            self.errorLog.append("ERROR: unable to find Instance document for {:s}".format(self.fname))
            return False
        
        ## 'findAll('us-gaap:earningspersharebasic')' doesn't seem to work... so find them manually
        self.all_tags = self.xbrlInstance.findAll()
        if not self.all_tags:
            self.errorLog.append("No tags found!")
            return False
        return True
    
    
    def save(self):
        """Saves the data to file."""
        pass
     
        
    @abstractmethod
    def getEps(self):
        """Retrieve the current EPS from the filing."""
        ## 'findAll('us-gaap:earningspersharebasic')' doesn't seem to work... so find them manually
        pass
            
        
    @abstractmethod
    def getSales(self):
        """Retrieves the current Sales data from the filing.
        
        WARNING! The 'currentContextRef' member must have been set first (e.g. by calling getEps or getRoe 
        before calling this function) for this function to work properly. It seems to be very tricky to 
        figure out the currentContextRef by itself."""
        pass
        
        
    @abstractmethod
    def getRoe(self):
        """Retrieves the current Return on Equity from the filing."""
        pass
        
    @abstractmethod
    def getCurrentValue(self, tagList):
        pass
    
            
    def getStartEndDateForContext(self, contextRef):
        """Looks up the date for the given contextref/id.

        This used to be a method under the SecFiling class. For some reason (me, Jupyter notebooks, other), 
        I would get an AttributeError that SecFiling10Q did not have this attribute when calling, e.g. 'getEps()
        on a SecFiling10Q object. So I nested it here under 'getCurrentValue'. """
        periodXml = self.xbrlInstance.find("xbrli:context", {"id" : contextRef})
        if periodXml:
            startdate = periodXml.find("xbrli:startdate") ## or just 'startdate'
            enddate = periodXml.find("xbrli:enddate") ## or just 'enddate'
            if not startdate and not enddate:
                startdate = periodXml.find("xbrli:instant") ## instantaneous date, not a date range
                enddate = startdate
        else:
            print("ERROR: Unable to find dates for contextref '{:s}'".format(contextRef))
            return None
        try:
            sd = datetime.strptime(startdate.text, "%Y-%m-%d")
        except:
            sd = None
        try:
            ed = datetime.strptime(enddate.text, "%Y-%m-%d")
        except:
            ed = None
        return [sd, ed]
        
        
    def printErrors(self):
        print("\n".join(e for e in self.errorLog))
            


In [3]:
class SecFiling10Q(SecFiling):
        
    def getEps(self):
        """Retrieve the current EPS from the filing."""
        all_eps_tags = []
        try:
            for tag in self.all_tags:
                if 'us-gaap:earningspersharebasic' in tag.name:
                    all_eps_tags.append(tag)
            self.currentEps = self.getCurrentValue(all_eps_tags)
        except:
            self.errorLog.append("Unable to find the EPS information in the filing.")
            return None
        return self.currentEps
                    
    
    def getSales(self):
        """Retrieves the current Sales data from the filing.
        
        WARNING! The 'currentContextRef' member must have been set first (e.g. by calling getEps or getRoe 
        before calling this function) for this function to work properly. It seems to be very tricky to 
        figure out the currentContextRef by itself."""
        if not self.currentContextId:
            print("ERROR! current contextId is not set!")
            return -99.0
        all_sales_tags = []
        try:
            for tag in self.all_tags:
                ## The filings seem to use either 'Revenues' or 'SalesRevenuesNet' to indicate the net sales amount
                ## But the 'Revenues' tag also contains other stuff, need to filter it by its contextref attribute
                if 'us-gaap:Revenues'.lower() == tag.name.strip():
                    if (tag.attrs)['contextref'] == self.currentContextId:
                        all_sales_tags.append(tag)
                elif 'us-gaap:SalesRevenuesNet'.lower() == tag.name.strip():
                    all_sales_tag.append(tag)
        except:
            self.errorLog.append("Unable to find Sales data in filing.")
            return None
        self.currentSales = self.getCurrentValue(all_sales_tags) 
        return self.currentSales
    
    
    def getRoe(self):
        """Retrieves the current Return on Equity from the filing."""
        ## First find the stockholders' equity
        all_se_tags = []
        ## Then find the net income
        all_ni_tags = []
        currentSE = None
        currentNI = None
        try:
            for tag in self.all_tags:
                if 'us-gaap:StockholdersEquity'.lower() == tag.name.strip():
                    all_se_tags.append(tag)
                if 'us-gaap:NetIncomeLoss'.lower() == tag.name.strip():
                    all_ni_tags.append(tag)
            currentSE = self.getCurrentValue(all_se_tags)
            currentNI = self.getCurrentValue(all_ni_tags)
        except:
            if not currentSE:
                self.errorLog.append("Unable to find Stockholders' Equity in filing.")
            if not currentNI:
                self.errorLog.append("Unable to find Net Income in filing.")
            return None
        ## Return on equity = net income / stockholders' equity
        self.currentRoe = 0.0
        if currentSE > 0.0:
            self.currentRoe = currentNI / currentSE
        return self.currentRoe
      
    
    def getCurrentValue(self, tagList):
        """Finds the tag for the current date from a list of tags, and returns its value."""
        prevDiff = timedelta(9999, 0, 0) ## days, seconds, microseconds (!)
        try:
            for tag in tagList:
                ## Find the quarter that ended closest to the reportDate, make sure it's a quarter
                ## TODO: figure out how to do this for the 10-K's
                contextRef = str((tag.attrs)['contextref'])
                if contextRef in self.contextIds:
                    tag_dates = self.contextIds[contextRef]
                else:
                    tag_dates = self.getStartEndDateForContext(contextRef)
                diff = self.reportDate - tag_dates[1]
                if abs(diff.days) < abs(prevDiff.days) and (tag_dates[1] - tag_dates[0]).days < 95:
                    current = tag
                    prevDiff = diff
            self.currentContextId = (current.attrs)['contextref']
        except Exception as ex:
            self.errorLog.append("Unable to determine the current value:")
            self.errorLog.append(type(ex).__name__)
            self.errorLog.append(ex)
            traceback.print_exc()
            return None
        return float(current.text)  
    
    def printErrors(self):
        super()


class SecFiling10K(SecFiling):
    def getEps(self):
        """Retrieve the current EPS from the filing."""
        all_eps_tags = []
        try:
            for tag in self.all_tags:
                if 'us-gaap:earningspersharebasic' in tag.name:
                    all_eps_tags.append(tag)
            self.currentEps = self.getCurrentValue(all_eps_tags)
        except:
            self.errorLog.append("Unable to find the EPS information in the filing.")
            return None
        return self.currentEps
                    
    
    def getSales(self):
        """Retrieves the current Sales data from the filing.
        
        WARNING! The 'currentContextRef' member must have been set first (e.g. by calling getEps or getRoe 
        before calling this function) for this function to work properly. It seems to be very tricky to 
        figure out the currentContextRef by itself."""
        if not self.currentContextId:
            print("ERROR! current contextId is not set!")
            return -99.0
        all_sales_tags = []
        try:
            for tag in self.all_tags:
                ## The filings seem to use either 'Revenues' or 'SalesRevenuesNet' to indicate the net sales amount
                ## But the 'Revenues' tag also contains other stuff, need to filter it by its contextref attribute
                if 'us-gaap:Revenues'.lower() == tag.name.strip():
                    if (tag.attrs)['contextref'] == self.currentContextId:
                        all_sales_tags.append(tag)
                elif 'us-gaap:SalesRevenuesNet'.lower() == tag.name.strip():
                    all_sales_tag.append(tag)
        except:
            self.errorLog.append("Unable to find Sales data in filing.")
            return None
        self.currentSales = self.getCurrentValue(all_sales_tags) 
        return self.currentSales
    
    
    def getRoe(self):
        """Retrieves the current Return on Equity from the filing."""
        ## First find the stockholders' equity
        all_se_tags = []
        ## Then find the net income
        all_ni_tags = []
        currentSE = None
        currentNI = None
        try:
            for tag in self.all_tags:
                if 'us-gaap:StockholdersEquity'.lower() == tag.name.strip():
                    all_se_tags.append(tag)
                if 'us-gaap:NetIncomeLoss'.lower() == tag.name.strip():
                    all_ni_tags.append(tag)
            currentSE = self.getCurrentValue(all_se_tags)
            currentNI = self.getCurrentValue(all_ni_tags)
        except:
            if not currentSE:
                self.errorLog.append("Unable to find Stockholders' Equity in filing.")
            if not currentNI:
                self.errorLog.append("Unable to find Net Income in filing.")
            return None
        ## Return on equity = net income / stockholders' equity
        self.currentRoe = 0.0
        if currentSE > 0.0:
            self.currentRoe = currentNI / currentSE
        return self.currentRoe

        
    def getCurrentValue(self, tagList):
        """Finds the tag for the current date from a list of tags, and returns its value."""
        prevDiff = timedelta(9999, 0, 0) ## days, seconds, microseconds (!)
        try:
            for tag in tagList:
                ## Find the quarter that ended closest to the reportDate, make sure it's a quarter
                ## TODO: figure out how to do this for the 10-K's
                contextRef = str((tag.attrs)['contextref'])
                if contextRef in self.contextIds:
                    tag_dates = self.contextIds[contextRef]
                else:
                    tag_dates = self.getStartEndDateForContext(contextRef)
                diff = self.reportDate - tag_dates[1]
                ## Make sure that enddate - startdate is about 365 days
                if abs(diff.days) < abs(prevDiff.days) and \
                        abs((tag_dates[1] - tag_dates[0]).days - 365) < 10:
                    current = tag
                    prevDiff = diff
            self.currentContextId = (current.attrs)['contextref']
        except Exception as ex:
            self.errorLog.append("Unable to determine the current value:")
            self.errorLog.append(type(ex).__name__)
            self.errorLog.append(ex)
            traceback.print_exc()
            return None
        return float(current.text)  



In [ ]:
import unittest

class TestNotebook(unittest.TestCase):

    def test_add(self):
        self.assertEqual(add(2, 2), 5)


unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
class CanslimParams(self):
    def __init__(self, ticker, all10QsDf, all10KsDf):
        self.ticker = ticker
        ## Format the 'date' column into datetime format
        all10QsDf.date.apply(pd.to_datetime)
        all10KsDf.date.apply(pd.to_datetime)
        self.all10QsDf = all10QsDf.sort_values('date', ascending=False)
        self.all10KsDf = all10KsDf.sort_values('date', ascending=False)
        self.today = datetime.now()
        self.fiveYearsAgo = self.today - timedelta(days=5*365.25)
        
    def loadData(self):
        """Loads the relevant SEC filings for analysis.
        
        Loads the last 4 10-K filings and the last 16(?) 10-Q filings. If necessary, 
        retrieves them from EDGAR and saves the raw files.
        """
        ## TODO: Look through the idx database and find all filings available for 'ticker'.
        ## TODO: Download/load each available filing in turn (=> list of SecFiling-classes)
        self.all10QFilings = {}
        self.all10KFilings = {}
        n10Qs = len(self.all10QsDf)
        n10Ks = len(self.all10KsDf)
        ## Create a dict of all the 10Q-filings objects
        for i in range(0, n10Qs):
            if (datetime.strptime(self.all10Qs.date.iloc[i]) > self.fiveYearsAgo):
                filing = SecFiling10Q(self.ticker)
                ## Download file if necessary, and generate the file name
                fname = filing.download(self.all10Qs.iloc[i].cik, self.all10Qs.iloc[i].conm, self.all10Qs.iloc[i].type, self.all10Qs.iloc[i].date, self.all10Qs.iloc[i].path)
                ## Load the file into the object instance
                filing.load(fname)
                ## Use the year+quarter (for filing date) information to create a key into the dict
                quarterKey = "{:d}-Q{:d}".format(self.all10Qs.iloc[i].date.year, int(self.all10Qs.iloc[i].date.month/4+1))
                print(quarterKey)
                self.all10QFilings[quarterKey] = filing
        
    
    def getEpsQuarter(self, quarter):
        """Returns the EPS for the specified quarter.
        
        The quarter is specified as an integer counting backwards, e.g. 0 (zero) is the current quarter,
        -1 (minus one) is the previous quarter, etc. For readability, the minus sign is required. Only 
        integers between -15 and 0 are allowed.
        """
        pass
    
    def getEpsAnnual(self, year):
        """Returns the EPS for the specified year.
        
        The year is specified as an integer counting backwards, e.g. 0 (zero) is the most recent reported year,
        -1 (minus one) is the previous year, etc. For readability, the minus sign is required. Only 
        integers between -3 and 0 are allowed.
        """
        pass
    
    def getRoeCurrent(self):
        """Returns the most recent Return on Equity."""
        pass
    
    def getSalesQuarter(self, quarter):
        """Returns the Sales for the specified quarter.
        
        The quarter is specified as an integer counting backwards, e.g. 0 (zero) is the current quarter,
        -1 (minus one) is the previous quarter, etc. For readability, the minus sign is required. Only 
        integers between -15 and 0 are allowed.
        """
        pass
    
    def getSalesAnnual(self, year):
        """Returns the Sales for the specified year.
        
        The year is specified as an integer counting backwards, e.g. 0 (zero) is the most recent reported year,
        -1 (minus one) is the previous year, etc. For readability, the minus sign is required. Only 
        integers between -3 and 0 are allowed.
        """
        pass
        
    def getEpsGrowthQuarter(self, q1, q2):
        """Calculates the EPS growth (%) for quarter q1 compared to q2.
        
        The EPS growth is calculated as the ratio EPS(q1)/EPS(q2) * 100%.
        """
        pass
    
    def getEpsGrowthAnnual(self, a1, a2):
        """Calculates the EPS growth (%) for year a1 compared to a2.
        
        The EPS growth is calculated as the ratio EPS(a1)/EPS(a2) * 100%.
        """
        pass
    
    def getStabilityOfEpsGrowth(self, numQuarters):
        """Calculates the stability of the quarterly EPS growth over the last numQuarters.
        
        The stability is calculated as the amount of deviation from the best-fit-line growth. 
        In other words, a line is fitted through the data, and the goodness-of-fit is determined.
        """
        pass
    
    def getEpsGrowthAcceleration(self, numQuarters):
        """Returns the (mean) acceleration of EPS growth over the specified number of quarters.
        
        The acceleration is calculated as the second derivative of the data. numQuarters is required to 
        be between 2 and 15. At least three quarters are necessary to calculate acceleration.
        """
        pass
    
    def getSalesGrowth(self, q1, q1):
        """Calculates the Sales growth (%) for quarter q1 compared to q2.
        
        The Sales growth is calculated as the ratio Sales(q1)/Sales(q2) * 100%.
        """
        pass
    
    def getSalesGrowthAcceleration(self, numQuarters):
        """Returns the (mean) acceleration of Sales growth over the specified number of quarters.
        
        The acceleration is calculated as the second derivative of the data. numQuarters is required to 
        be between 2 and 15. At least three quarters are necessary to calculate acceleration.
        """
        pass
    
    def plotEpsQuarter(self):
        """Generates a log-plot of quarterly EPS data."""
        pass
    
    def plotStockData(self):
        """Generates a plot of the weekly stock data for the last three years."""
        pass
    
    def getStockData(self):
        """Download the weekly stock data for the last three years from somehwere."""
        pass
    
    def getStockGrowth(self):
        """Returns the stock growth as the slope of the best-fit line through the stock data."""
        pass
    
    def getStockAcceleration(self):
        """Fits the equation a*x^2+b*x+c through the data and returns the 'a' coefficient."""
        pass
    
    ## Lofty future goal: write algorithm(s) that identifies Canslim patterns in the stock data
        

In [64]:
all_10Qs.date.apply(pd.to_datetime)
print(all_10Qs.iloc[1].date.month/4+1)
print(pd.PeriodIndex(all_10Qs.iloc[1:2].date, freq='Q'))
all_10Qs.iloc[2].date.month

3.0
PeriodIndex(['2014Q3'], dtype='period[Q-DEC]', name='date', freq='Q-DEC')


11

In [19]:
ticker= "NVDA"
import pandas
from sqlalchemy import create_engine

## Open the database of index file names to generate url
## Note: Loading this database takes a long time! Do this once, and pass the cik_ticker_name 
## dataframe to the CanslimParams class
engine = create_engine('sqlite:///edgar_idx.db')
with engine.connect() as conn, conn.begin():
    #load the table with the index files info
    idx = pandas.read_sql_table('idx', conn)
    #load the look-up table for ticker symbol to CIK translation
    cik_ticker_name = pandas.read_sql_table ('cik_ticker_name', conn)
    #handle the case where there are multiple stocks with the same ticker; just select the first?!?
    cik =((cik_ticker_name.cik[cik_ticker_name.ticker == ticker]))
    print ((cik.iloc[0]))
    all_links = idx[idx.cik == cik.iloc[0]]
    all_10Qs = all_links[all_links.type == '10-Q']
    #verify that this gets the amended 10-Q's
    #all_10Qs.append (all_links[all_links.type == '10-Q\A'])
    all_10Ks = all_links[all_links.type == '10-K']
    print (all_10Qs, all_10Ks)

1045810
             cik         conm  type        date  \
325292   1045810  NVIDIA CORP  10-Q  2014-05-21   
573999   1045810  NVIDIA CORP  10-Q  2014-08-20   
787113   1045810  NVIDIA CORP  10-Q  2014-11-19   
1328069  1045810  NVIDIA CORP  10-Q  2015-05-20   
1584527  1045810  NVIDIA CORP  10-Q  2015-08-19   
1792044  1045810  NVIDIA CORP  10-Q  2015-11-18   
2313433  1045810  NVIDIA CORP  10-Q  2016-05-25   
2553929  1045810  NVIDIA CORP  10-Q  2016-08-23   
2755924  1045810  NVIDIA CORP  10-Q  2016-11-22   
3269421  1045810  NVIDIA CORP  10-Q  2017-05-23   
3521777  1045810  NVIDIA CORP  10-Q  2017-08-23   
3721589  1045810  NVIDIA CORP  10-Q  2017-11-21   
4232531  1045810  NVIDIA CORP  10-Q  2018-05-22   
4477972  1045810  NVIDIA CORP  10-Q  2018-08-16   

                                                path  
325292   edgar/data/1045810/0001045810-14-000085.txt  
573999   edgar/data/1045810/0001045810-14-000158.txt  
787113   edgar/data/1045810/0001045810-14-000188.txt  
132806

In [37]:
all_10Qs.iloc[2].cik

'1045810'

In [ ]:
cik =((cik_ticker_name.cik[cik_ticker_name.ticker == "NVDA"]))
print ((cik.iloc[0]))
all_links = idx[idx.cik == cik.iloc[0]]
all_10Qs = all_links[all_links.type == '10-Q']
#verify that this gets the amended 10-Q's
#all_10Qs.append (all_links[all_links.type == '10-Q\A'])
all_10Ks = all_links[all_links.type == '10-K']
print (all_10Qs.iloc[10], all_10Ks.iloc[1])

In [4]:
nvda_10q = SecFiling10Q("NVDA")
#nvda_10q.download(all_10Qs.iloc[10].cik, all_10Qs.iloc[10].conm, all_10Qs.iloc[10].type, all_10Qs.iloc[10].date, all_10Qs.iloc[10].path)
## Note: Loading takes a long time.
nvda_10q.load('SECDATA\\NVIDIA CORP\\1045810_NVIDIA CORP_10-Q_2017-08-23')
#nvda_10q.getDateForContext("FD2017Q2QTD")
#nvda_10q.printErrors()
print(nvda_10q.getEps())
print(nvda_10q.getSales())
print(nvda_10q.getRoe())
nvda_10q.printErrors()

Report date: 2017-07-30 00:00:00, industrial class: SEMICONDUCTORS & RELATED DEVICES [3674]


Traceback (most recent call last):
  File "<ipython-input-3-35d71294c159>", line 83, in getCurrentValue
    tag_dates = self.getStartEndDateForContext(contextRef)
TypeError: getStartEndDateForContext() takes 1 positional argument but 2 were given
Traceback (most recent call last):
  File "<ipython-input-3-35d71294c159>", line 83, in getCurrentValue
    tag_dates = self.getStartEndDateForContext(contextRef)
TypeError: getStartEndDateForContext() takes 1 positional argument but 2 were given
Traceback (most recent call last):
  File "<ipython-input-3-35d71294c159>", line 83, in getCurrentValue
    tag_dates = self.getStartEndDateForContext(contextRef)
TypeError: getStartEndDateForContext() takes 1 positional argument but 2 were given


None
ERROR! current contextId is not set!
-99.0


TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [ ]:
all_tags = nvda_10q.xbrlInstance.findAll()
all_se_tags = []
count=0
for t in all_tags:
    count+=1
    if 'us-gaap:StockholdersEquity'.lower() == t.name.strip():
        all_se_tags.append(tag)
print(count)

In [ ]:
(all_se_tags[0])

In [ ]:
            period = xbrlInstance.find("xbrli:context", {"id" : contextref})
            if period:
                startdate = period.find("xbrli:startdate") ## or just 'startdate'
                enddate = period.find("xbrli:enddate") ## or just 'enddate'
                self.contextIds[contextref] = \
                    [datetime.strptime(startdate.text, "%Y-%m-%d"), datetime.strptime(enddate.text, "%Y-%m-%d")]
                print("Contextref {:s} found:\n{:s} to {:s}"\
                      .format(contextRef, str(self.contextIds[contextRef][0]), str(self.contextIds[contextRef][1])))

## Candidate keywords  
### Shareholder's equity  
*  FinancialSupportCapitalContributionsMember
*  StatementOfStockholdersEquityAbstract
*  StockholdersEquity
### EPS/Earnings/Profits/Income  
*  EarningsPerShareBasic
*  EarningsPerShareDiluted
*  IncomeLossFromContinuingOperationsPerDilutedShare(?)
*  IncomeLossFromContinuingOperationsPerBasicShare (?)
### Sales/Net sales/net revenue  
*  SalesRevenueNetMember (?)
*  Search for 'Revenue' in income statement (?)
### Others that might be of interest  
*  us-gaap:StockRepurchasedDuringPeriodShares


# Data to extract from filing:
*  Earnings for current Q
*  Earnings for last 12-20 Q
*  Sales
*  Annual EPS for last 4 years
*  ROE (= net income / shareholder's equity)
# Parameters to calculate for each company:
*  Earnings increase: earning for current Q/earnings for same Q prior year
*  Excellency of earnings increase: each of two most recent Q/same Q's prior year + growth rate over last three years
*  Earnings growth acceleration in last 10 Q (calculate 2nd derivative?)
   *  two consecutive Qs of earnings *deceleration* could mean trouble
   *  plot earnings on log-scale for last 12 months to see acceleration
*  Sales increase: 
   *  current Q/prior Q > 25%
   *  *or* Sales growth is accelerating in the last three Q
*  Acceleration of sales growth and earnings growth in last three Q (don't sell if accelerating!)
*  Annual EPS should be increasing in each of the last three years.
*  ROE>17%
*  Stability of Q-EPS over last 3-5 years (plot EPS, fit line through data to determine growth trend, calculate deviation of EPS's from this growth trend (i.e. goodness of fit))

In [ ]:
import requests
from bs4 import BeautifulSoup as Bsoup
import pandas as pd

In [ ]:
ticker = "NVDA"
## Read in the file
f = open ("1045810_NVIDIA-CORP_10-Q_2017-08-23", "r")
lines = f.readlines ()
f.close ()
ser = pd.Series (lines)
#print (ser)
#this denotes where the actual 10-Q starts. I think the 10-Q part is enclosed by <document> tags
html_idxs = (ser[ser == '<TYPE>10-Q\n'])
#get the line number (== index number):
print ("html_idxs: {}".format(html_idxs.index.values))
#and the corresponding line
print (lines[html_idxs.index.values[0]])
#html_idxs_end = (ser['<\TYPE>' in ser])
#print ("html_idxs_end: {}".format (html_idxs_end.index.values))
#print (lines[html_idxs_end.index.values[0]])

#now I can parse the 10-Q with beautifulsoup:
## Find a way to determine which lines to parse automatically. Really, I only need the html-tag, i.e. lines 55-63 
## (see NVDA_finstat_test.hmtl)
soup = Bsoup ("".join (lines[55:63]), "lxml") #find the end of the section by searching for /Document?
#collect all div-tags for future use
all_divs = soup.find_all ("div")
#find the div-tag that contains 'ITEM 1.&#160;&#160;FINANCIAL STATEMENTS' which indicates the start of the
#income statement's table
#will this work on all filings???

print ("found {} occurrences". format (len(all_divs)))
tables = soup.find_all ("table")
print ("found {} tables".format (len(tables)))
#iterate over the div-tags, not sure if there is a better way. I think everything is is div tags.
flag = False
count = 0
for f in all_divs:
    #find beginning of financial statements
    if (f.get_text().startswith ("ITEM 1.") and "FINANCIAL STATEMENTS" in f.get_text()):
        flag = True
        #print (count)
    if (flag):
        #print (f.get_text())
        if ("three months ended" in f.text.lower()):
            ## At this point, we found the div-tag with the income-statement table
            ## Try to use pandas
            #print((f.find_all('div', text='$')))
            for ff in f.find_all('div',text='$'):
                ff.decompose()
            ## Almost! Just need to get rid of the '$', which are offsetting the affected rows by several columns
            ## This removes the '$' (use re.compile for other currency symbols?), now what about the paren's denoting negative numbers?
            tableDf=pd.read_html(str(f))
            print(tableDf)
            ## Manual method
            trs = f.find_all("tr")
            #print (len(trs))
            #print(trs[2].get_text())
            for t in trs:
                tlist = [item.get_text() for item in t.find_all("td")]
                print(" || ".join(tlist))
            #print (trs)
            break
        #print ("\n")
    count += 1



### See these urls for downloading the appropriate files, and parsing XBRL with Python/BeautifulSoup
*  https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python
*  https://www.slideshare.net/afalk42/xbrl-us-altova-webinar
### Taxonomy:
*  http://www.xbrlsite.com/LinkedData/BrowseObjectsByType_HTML.aspx?Type=%5BConcept%5D&Submit=Submit
*  ~~I think I want the instance xml file for each filing,~~ though that \*.txt file I've been downloading so far contains *everything* (I think). Could use that as well, the benefits are that the name of this file is standardized and I already have the machinery to download it in place, but the trade-off is that it's the biggest file for each filing. 
   *  The instance document can be found in the \*.txt file; it has the `<description>` XBRL INSTANCE DOCUMENT. Inside the description-tag is the `<XBRL>`, which contains the XBRL-data for the instance document. Note that there are several XBRL- tags containing the various XBRL documents (schema, extension, linkbases, ...).  
*  Search for the tags 'us-gaap:\*', they contain the items for the financial statements.
*  Also, Wikipedia has a good reference on the components of XBRL: 
   *  https://en.wikipedia.org/wiki/XBRL
### To do:
-  Figure out which taxonomy items I want/need, and all their permutations
-  Determine the date on each item
   -  Each us-gaap:\* - tag has a context-ref (e.g. "FI2017Q4"). Find the corresponding context (in the instance document, below the us-gaap - tags):
      ```
      <xbrli:context id="FI2017Q4">
		<xbrli:entity>
			<xbrli:identifier scheme="http://www.sec.gov/CIK">0001045810</xbrli:identifier>
		</xbrli:entity>
		<xbrli:period>
			<xbrli:instant>2017-01-29</xbrli:instant>
		</xbrli:period>
	</xbrli:context>
    ```  
### Interesting classifiers:
-  unitref=
